In [1]:
import pickle
import pandas as pd
from pyspark import SparkContext
import time
sc = SparkContext.getOrCreate()

In [ ]:
def process_data(raw_data):
    freq = 700
    wrist_data = raw_data['signal']['wrist']
    signal_keys = wrist_data.keys()
    df = pd.DataFrame(raw_data['label'], columns=['label'])
    df.index = [(1 / freqs['label']) * i for i in range(len(df))]
    for mod in signal_keys:
        if mod=='ACC':
            for i in range(3):
                df2 = pd.DataFrame(wrist_data[mod][:, i], 
                                   columns=[mod+f'_{i}'])
                df2.index = [(1 / freqs[mod]) * i for i in range(len(df2))]
                df = df.join(df2, how='inner')
        else:
            df2 = pd.DataFrame(wrist_data[mod], columns=[mod])
            df2.index = [(1 / freqs[mod]) * i for i in range(len(df2))]
            df = df.join(df2, how='inner')
    return df

In [17]:
all_chest_tables = pd.DataFrame()
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]


def load_data(fname):
    with open(fname, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        raw_data = u.load()
    return raw_data

def get_chest_data(raw_data):
    raw_chest_data = raw_data['signal']['chest']
    key_chest = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
    subject = raw_data['subject']
    labels = raw_data['label']
    df = pd.DataFrame(labels, columns=['label'])
    df['sub_id'] = subject
    for key in key_chest:
        if key=='ACC':
            df[f'chest_{key}_1'] = raw_chest_data[key][:, 0]
            df[f'chest_{key}_2'] = raw_chest_data[key][:, 1]          
            df[f'chest_{key}_3'] = raw_chest_data[key][:, 2]
        else:
            df[f'chest_{key}'] = raw_chest_data[key]
    return df

def add_time_col(df, freq=700):
    df['col_1'] = [(1 / freq) * i for i in range(len(df))]
    return df

In [18]:
data_path = "/Users/alaa/data/WESAD/"

In [19]:
for i in nums:
    fname = data_path+f"S{i}/S{i}.pkl"
    raw_data = load_data(fname)
    df = get_chest_data(raw_data)
    df = add_time_col(df)
    df.to_csv(f"Chest/S_{i}.csv")